# Notebook 03. Introduction to Ray Tune and hyperparameter optimization (HPO)

© 2019-2022, Anyscale. All Rights Reserved <br>
📖 [Back to Table of Contents](./ex_00_rllib_notebooks_table_of_contents.ipynb) <br>

➡️ [Next notebook](./ex_04_offline_rl_with_rllib.ipynb) <br>
⬅️ [Previous notebook](./ex_02_create_multiagent_rllib_env.ipynb) <br>

### Learning objectives
In this this notebook, you will learn:
 * [How to configure Ray Tune to find solid hyperparameters more easily](#configure_ray_tune)
 * [The details behind Ray RLlib resource allocation](#resource_allocation)
 

In [1]:
# Import required packages.

import gym
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray import tune

# Importing the very same environment class that we have coded together in
# the previous notebook.
from multi_agent_arena.multi_agent_arena import MultiAgentArena


print(f"gym: {gym.__version__}")
print(f"ray: {ray.__version__}")

# !ale-import-roms --import-from-pkg atari_py.atari_roms

gym: 0.21.0
ray: 3.0.0.dev0


### How to configure Ray Tune to find solid hyperparameters more easily <a class="anchor" id="configure_ray_tune"></a>

In the previous experiments, we used a single algorithm's (PPO) configuration to create
exactly one Algorithm object and call its `train()` method manually a couple of times.

A common thing to try when doing ML or RL is to look for better choices of hyperparameters, neural network architectures, or algorithm settings. This hyperparameter optimization
problem can be tackled in a scalable fashion using Ray Tune (in combination with RLlib!).

<img src="images/rllib_and_tune.png" width=800>


The following cell demonstrates, how you can setup a simple grid-search for one very important hyperparameter (the learning rate), using our already existing PPO config object and Ray Tune:

In [4]:
# Create a PPOConfig object (same as we did in the previous notebook):
config = PPOConfig()

# Setup our config object the exact same way as before:
# Point to our MultiAgentArena env:
config.environment(env=MultiAgentArena)

# Setup multi-agent mapping:

# Environment provides M agent IDs.
# RLlib has N policies (neural networks).
# The `policy_mapping_fn` maps M agent IDs to N policies (M <= N).

# If you don't provide a policy_mapping_fn, all agent IDs will map to "default_policy".
config.multi_agent(
    # Tell RLlib to create 2 policies with these IDs here:
    policies=["policy1", "policy2"],
    # Tell RLlib to map agent1 to policy1 and agent2 to policy2.
    policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "policy1" if agent_id == "agent1" else "policy2",
)

# Reduce the number of workers from 2 (default) to 1 to save some resources on the expensive hyperparameter sweep.
# IMPORTANT: More information on resource requirements for tune hyperparameter sweeps and different RLlib algorithm setups
# below.
config.rollouts(num_rollout_workers=1)

Now, let's explore how a very simple hyperparameter search should be configured with RLlib and Tune:

In [5]:
# Before setting up the learning rate hyperparam sweep,
# let's see what the default learning rate and train batch size is for PPO:
print(f"Default learning rate for PPO is: {config.lr}")
print(f"Default train batch size for PPO is: {config.train_batch_size}")

Default learning rate for PPO is: 5e-05
Default train batch size for PPO is: 4000


In [7]:
# Now let's change our existing config object and add a simple
# grid-search over two different learning rates to it:
config.training(
    lr=tune.grid_search([5e-5, 1e-4]),
    train_batch_size=tune.grid_search([3000, 4000]),
)


💡 <b>Right-click on the cell below and choose "Enable Scrolling for Outputs"!</b>  This will make it easier to view, since model training output can be very long!

In [8]:
# Example using Ray tune API (`tune.run()`) until some stopping condition is met.
# This will create one (or more) Algorithms under the hood automatically w/o us having to
# build these algos from the config.

experiment_results = tune.run(
    "PPO",

    # training config params (translated into a python dict!)
    config=config.to_dict(),

    # Stopping criteria whichever occurs first: average reward over training episodes, or ...
    stop={
        "training_iteration": 6,     # stop after n training iterations (calls to `Algorithm.train()`)
        #"episode_reward_mean": 400, # stop if average (sum of) rewards in an episode is 400 or more
        #"timesteps_total": 100000,  # stop if reached 100,000 sampling timesteps
    },  

    # redirect logs instead of default ~/ray_results/
    local_dir="results",
         
    # Every how many train() calls do we create a checkpoint?
    checkpoint_freq=1,
    # Always save last checkpoint (no matter the frequency).
    checkpoint_at_end=True,

    ###############
    # Note about Ray Tune verbosity.
    # Screen verbosity in Ray Tune is defined as verbose = 0, 1, 2, or 3, where:
    # 0 = silent
    # 1 = only status updates, no logging messages
    # 2 = status and brief trial results, includes logging messages
    # 3 = status and detailed trial results, includes logging messages
    # Defaults to 3.
    ###############
    verbose=3,
                   
    # Define what we are comparing for, when we search for the
    # "best" checkpoint at the end.
    metric="episode_reward_mean",
    mode="max",
)

print("Training completed.")


Usage stats collection is enabled. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


2022-08-09 12:40:46,589	INFO worker.py:1481 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265.


Trial name,status,loc,lr,train_batch_size,iter,total time (s),ts,reward,num_recreated_wor...,episode_reward_max,episode_reward_min
PPO_MultiAgentArena_bafe8_00000,TERMINATED,127.0.0.1:63785,5e-05,3000,6,72.961,18000,-0.612,0,19.5,-18
PPO_MultiAgentArena_bafe8_00001,TERMINATED,127.0.0.1:63794,0.0001,3000,6,74.022,18000,1.134,0,19.8,-18.3
PPO_MultiAgentArena_bafe8_00002,TERMINATED,127.0.0.1:63797,5e-05,4000,6,94.7767,24000,-1.152,0,19.8,-25.5
PPO_MultiAgentArena_bafe8_00003,TERMINATED,127.0.0.1:63801,0.0001,4000,6,94.4711,24000,2.727,0,19.5,-18


(PPO pid=63785) 2022-08-09 12:40:57,092	INFO algorithm.py:1871 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPO pid=63785) 2022-08-09 12:40:57,093	INFO algorithm.py:351 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=63792) 2022-08-09 12:41:04,490	WARNING env.py:235 -- Your MultiAgentEnv <MultiAgentArena instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__` from within your MutiAgentEnv's constructor. This will raise an error in the future.
(PPO pid=63785) 2022-08-09 12:41:07,173	INFO trainable.py:160 -- Trainable.setup took 10.083 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation o

Result for PPO_MultiAgentArena_bafe8_00000:
  agent_timesteps_total: 6000
  counters:
    num_agent_steps_sampled: 6000
    num_agent_steps_trained: 6000
    num_env_steps_sampled: 3000
    num_env_steps_trained: 3000
  custom_metrics: {}
  date: 2022-08-09_12-41-16
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 9.300000000000011
  episode_reward_mean: -11.360000000000003
  episode_reward_min: -31.50000000000007
  episodes_this_iter: 30
  episodes_total: 30
  experiment_id: 4f1310ef73844c1c99a53824a8d7ce26
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 4.999999873689376e-05
          entropy: 1.373630404472351
          entropy_coeff: 0.0
          kl: 0.0128685487434268
          model: {}
          policy_loss: -0.030643977224826813
          total_loss: 7.721398830413818
          vf_explained_var: -0.0014218075666576624
          vf_loss: 

...

Result for PPO_MultiAgentArena_bafe8_00001:
  agent_timesteps_total: 6000
  counters:
    num_agent_steps_sampled: 6000
    num_agent_steps_trained: 6000
    num_env_steps_sampled: 3000
    num_env_steps_trained: 3000
  custom_metrics: {}
  date: 2022-08-09_12-41-34
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 29.399999999999977
  episode_reward_mean: -11.070000000000006
  episode_reward_min: -28.80000000000005
  episodes_this_iter: 30
  episodes_total: 30
  experiment_id: ba802e21773d46369dab10c339d8b4ae
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 9.999999747378752e-05
          entropy: 1.3686600923538208
          entropy_coeff: 0.0
          kl: 0.01781587116420269
          model: {}
          policy_loss: -0.04294942691922188
          total_loss: 7.1038126945495605
          vf_explained_var: 0.013475432991981506
          vf_loss:

(PPO pid=63801) 2022-08-09 12:42:13,267	WARNING deprecation.py:47 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Result for PPO_MultiAgentArena_bafe8_00000:
  agent_timesteps_total: 12000
  counters:
    num_agent_steps_sampled: 12000
    num_agent_steps_trained: 12000
    num_env_steps_sampled: 6000
    num_env_steps_trained: 6000
  custom_metrics: {}
  date: 2022-08-09_12-42-15
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 24.299999999999947
  episode_reward_mean: -5.254999999999996
  episode_reward_min: -31.50000000000007
  episodes_this_iter: 30
  episodes_total: 60
  experiment_id: 4f1310ef73844c1c99a53824a8d7ce26
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      policy1:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 4.999999873689376e-05
          entropy: 1.3507912158966064
          entropy_coeff: 0.0
          kl: 0.007976443506777287
          model: {}
          policy_loss: -0.031042877584695816
          total_loss: 7.243046283721924
          vf_explained_var: 0.014940924011170864
          vf_lo

2022-08-09 12:43:39,441	INFO tune.py:758 -- Total run time: 171.47 seconds (170.68 seconds for the tuning loop).


Training completed.


In [10]:
# Using the returned `experiment_results` object,
# we can extract from it the best checkpoint according to some criterium, e.g. `episode_reward_mean`.

# We only had a single trial (one Algorithm instance), so this should be returned here.
best_trial = experiment_results.get_best_trial()
print("Best trial: ", best_trial)


# From that trial, extract the best checkpoint (max `evaluation/episode_reward_mean` value).
best_checkpoint = experiment_results.get_best_checkpoint(trial=best_trial, metric="episode_reward_mean", mode="max")

# We would expect this to be either the very last checkpoint or one close to it:
print(f"Best checkpoint from training: {best_checkpoint}")

Best trial:  PPO_MultiAgentArena_bafe8_00003
Best checkpoint from training: Checkpoint(local_path=/Users/sven/Dropbox/Projects/ray-summit-2022-training/ray-rllib/results/PPO/PPO_MultiAgentArena_bafe8_00003_3_lr=0.0001,train_batch_size=4000_2022-08-09_12-41-44/checkpoint_000006)


### The details behind Ray RLlib resource allocation <a class="anchor" id="resource_allocation"></a>

#### Why did we use 8 CPUs in the tune run above (2 CPUs per trial)?

```
== Status ==
Current time: 2022-07-24 18:18:28 (running for 00:02:09.35)
Memory usage on this node: 9.9/16.0 GiB
Using FIFO scheduling algorithm.
Resources requested: 8/16 CPUs, 0/0 GPUs, 0.5/3.97 GiB heap, 0.5/1.98 GiB objects
```

<img src="images/closer_look_at_rllib.png" width=700 />

By default, the PPO Algorithm uses 2 so called `RolloutWorkers` (you can change this via `config.rollouts(num_rollout_workers=2)`) for collecting samples from
environments in parallel.
We changed this setting to only 1 worker via the `config.rollouts(num_rollout_workers=1)` call in the cell above.

`RolloutWorkers` are Ray Actors that have their own copies of the environment and step through episodes in parallel. Each Actor in Ray normally uses a single CPU, but besides `RolloutWorker`s, an Algorithm in RLlib also always has one local process (aka. the "driver" process or the "local worker"), which - in case of PPO -
handles the model/policy learning updates.

For our experiment above, this gives us 2 CPUs (1 rollout worker + 1 local learner) per Algorithm instance.

Since our config specifies two `grid_search` with 2 different learning rates AND 2 different batch sizes, we were running 4 Algorithms in parallel above (2 learning rates x 2 batch sizes = 4 trials), hence 8 CPUs were required (4 algos x 2 CPUs each = 8).


### Summary

In this notebook, we have learnt, how to:

* Use Ray Tune in combination with RLlib for hyperparameter tuning
* How RLlib and Tune determine the required computational resources for some `tune.run()` experiment

### Exercise 03<a ></a>

#### Using the `config` that we have built so far, let's run another `tune.run()`.

But this time, apply the following changes to our setup:

- Setup only 1 learning rate using the `config.training(lr=...)` method call. Chose the (seemingly) best value from the run in the previous cell (the one that yielded the highest avg. reward).
- Setup only 1 train batch size using the `config.training(train_batch_size=...)` method call. Chose the (seemingly) best value from the run in the previous cell (the one that yielded the highest avg. reward).
- Set the number of RolloutWorkers to 5 using the `config.rollouts(num_rollout_workers=5)` method call, which will allow us to collect more environment samples in parallel.
- Set the `num_envs_per_worker` config parameter to 5 using the `config.rollouts(num_envs_per_worker=...)` method call. This will batch our environment on each rollout worker, and thus parallelize action computing forward passes through our neural networks.

Other than that, use the exact same args as in our `tune.run()` call in the previous cell.

**Good luck! :)**

 ## References
 * [Tune, Scalable Hyperparameter Tuning](https://docs.ray.io/en/latest/tune/index.html)

⬅️ [Previous notebook](./ex_02_create_multiagent_rllib_env.ipynb) <br>
➡️ [Next notebook](./ex_04_offline_rl_with_rllib.ipynb) <br>

📖 [Back to Table of Contents](./ex_00_rllib_notebooks_table_of_contents.ipynb)<br>